##### Installing Required Libraries : 

EUMDAC library will be installed. EUMDAC is the EUMETSAT Data Access Client. It provides simple access to the EUMETSAT data of all satellite missions. As a Python library, it comes with many methods and helpers to use EUMETSATs APIs and services, like Data Store. 

In [ ]:
# ! pip install eumdac
# ! pip install requests
# ! pip install patool
# ! pip install satpy
# ! pip install rasterio
# ! pip install matplotlib
# ! pip install cartopy 
! wget -N -O utils.py 'https://github.com/Simoniman/GeoSatStitch/raw/main/utils.py' 
! wget -N -O config.py 'https://github.com/Simoniman/GeoSatStitch/raw/main/config.py' 

##### Importing Required Libraries

In [ ]:
import os
import time
import datetime
import shutil
from glob import glob
import requests
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import eumdac
import patoolib
from satpy import Scene
from pyresample import create_area_def
from utils import find_format, find_reader, geo_area_def, extract_zip_files, read_credentials_from_config

##### Downloading latest files

Insert your personal key and secret into the single quotes. https://eoportal.eumetsat.int/

In [ ]:
consumer_key, consumer_secret = read_credentials_from_config(filename='../config.json')

credentials = (consumer_key, consumer_secret)

token = eumdac.AccessToken(credentials)

1-Discovering **Collections**

collection identifier - collection title
  - EO:EUM:DAT:MSG:HRSEVIRI - High Rate SEVIRI Level 1.5 Image Data - MSG - 0 degree 
  - EO:EUM:DAT:MSG:HRSEVIRI-IODC - High Rate SEVIRI Level 1.5 Image Data - MSG - Indian Ocean


In [ ]:
seviri_pdcid = "EO:EUM:DAT:MSG:HRSEVIRI" # prime data collection id
seviri_iodcid = "EO:EUM:DAT:MSG:HRSEVIRI-IODC" # indian ocean data collection id

Data Collections based on collection identifiers

In [ ]:
datastore = eumdac.DataStore(token)
seviri_pdc = datastore.get_collection(seviri_pdcid) # prime data collection
seviri_iodc = datastore.get_collection(seviri_iodcid) # indian ocean data collection

2-Searching and Filtering to find required **Products**/**Datasets**

Filtering by latest product

In [ ]:
seviri_pp_latest = seviri_pdc.search().first() # prime product_latest
seviri_iop_latest = seviri_iodc.search().first() # indian ocean product_latest
products = [seviri_pp_latest, seviri_iop_latest]

3-Downloading **Products**

 Products can be downloaded by providing either their product ID, or a combination of their collection ID and the time of acquisition. We can download entire products, or specific file components (e.g. metadata only).

In [ ]:
# Create the "tmp" directory if it doesn't exist
base_dir = './Meteosat_seviri/tmp'
os.makedirs(base_dir, exist_ok=True)

for product in products:
    try:
        with product.open() as fsrc, open(os.path.join(base_dir, os.path.basename(fsrc.name)), mode='wb') as fdst:
            shutil.copyfileobj(fsrc, fdst)
            print(f'Download of product {product} finished.')
    except eumdac.product.ProductError as error:
        print(f"Error related to the product '{product}' while trying to download it: '{error.msg}'")
    except requests.exceptions.ConnectionError as error:
        print(f"Error related to the connection: '{error.msg}'")
    except requests.exceptions.RequestException as error:
        print(f"Unexpected error: {error}")
    
print('All downloads are finished.')

Then we will extract archived file.

In [ ]:
extract_zip_files(base_dir)

##### Reading Data

Satpy's main interface for working with data is the `Scene` class. We can provide the `Scene` with data files and load them with a "reader". 

There are many readers available in Satpy, but the three readers related to this project are as below :

| Reader      | Description |
|:-----------:|:-----------:|
| `'abi_l1b'`              |  reads individual GOES ABI L1B NetCDF4 files. The files read by this reader are described in the [official PUG document](https://www.goes-r.gov/users/docs/PUG-L1b-vol3.pdf)|
| `'seviri_l1b_native'`    |  reads and calibrates MSG-SEVIRI L1.5 image data in binary format. The format is explained in the [MSG Level 1.5 Native Format File Definition.](https://www-cdn.eumetsat.int/files/2020-04/pdf_fg15_msg-native-format-15.pdf)|
|   `'ahi_hsd'`            |  reads Advanced Himawari Imager (AHI) Binary data (standard format data)       |

In [ ]:
filenames_format = find_format(base_dir)

reader=find_reader(filenames_format)
filenames = glob( base_dir + f'/*.{filenames_format}')

scn = Scene(reader=reader, filenames=filenames)

We've now created a `Scene` object. Under the hood Satpy has determined what we can access. We haven't actually loaded any data so our dict-like `Scene` object is empty. In other words we first load/read metadata by creating `Scene` object. 

In [ ]:
scn.available_dataset_names()

Then we will read/load all or subset of available dataset. 

In [ ]:
channels = ['IR_108'] 
scn.load(channels)

Now we loaded some dataset and consequently are able to access those.

In [ ]:
scn['IR_108']

##### Resampling Data

One of the more complex topics when it comes to working with earth-observing satellite data is geographic projections and resampling data to these different projections.

The `'area'` attribute of our Satpy data is a special `AreaDefinition` and it defines the geographic area that our data covers. Under `Projection` we see a python dictionary of projection parameters to define that flat plane representation of the Earth. Our ABI data is on a `'geos'` or Geostationary Satellite View coordinate system where position is measured in meters on the the X and Y axes. You can learn more about that on the PROJ site [here](https://proj.org/operations/projections/geos.html).

In the current project we need to have the data in `'web mercator'` projection system(EPSG:3857).
So we need to resample the data to this system.  

We can do this using Satpy's `Scene.resample` method which provides multiple algorithms for resampling data.

- `'native'` resampler can be very useful when you need to combine bands from the same instrument that have different resolutions.

- Other resamplers (`nearest` and so on) are used to change the projection of the dataset. for this case, we should determine the geographic area of interest using a `AreaDefenition` object. this object is created more easily using Pyresample's `create_area_def` utility function.

Pysat utilizes Pyresample, a Python package designed for resampling geospatial image data, for the purpose of resampling operations.

Next we will create `AreaDefenition` object for Goes_East and Goes_West satellites using create_area_def utility function. 

In [ ]:
geo_area_defenition = geo_area_def(scene=scn)

Then we will resample the data using created `AreaDefenition` object. 

In [ ]:
scn_resamp = scn.resample(destination=geo_area_defenition)

##### Writing Data

Satpy makes it possible to save datasets in multiple formats, with `writers` designed to save in a given format.The default `writer` is geotiff writer.

In [ ]:
scn_resamp.save_datasets(
    writer="geotiff",
    filename="{platform_name}_{name}_{start_time:%Y%m%d_%H%M%S}.tif",
    base_dir=base_dir)

##### Visualizing Data

Although data can be visualized in specialized GIS software after exporting the data in well-known geospatial raster formats like GeoTIFF, being able to visualize data directly in Python can be really beneficial in some cases.

In [ ]:
channel = 'C06'
crs = scn[channel].attrs['area'].to_cartopy_crs()
plt.figure(figsize=(16,10))
ax = plt.axes(projection=ccrs.PlateCarree())

cmap = 'Greys'  # Example colormap without transparency

dataset = scn[channel]
dataset.plot.imshow(transform=crs, cmap=cmap)
ax.coastlines()
# ax.gridlines()
ax.gridlines(draw_labels=True, linewidth=1, color='gray', alpha=0.5, linestyle='--')
plt.title(f"platform_name : {scn[channel].attrs['platform_name']}  \n channel : {scn[channel].attrs['name']}  \n start_time : {scn[channel].attrs['start_time']}  \n end_time : {scn[channel].attrs['end_time']} ")